In [1]:
import sys
sys.path.append('../')
import newtable
import pprint

In [2]:
core = newtable.ConnectCore.open(target=':memory:', dialect='sqlite')
core

ConnectCore(target=':memory:', dialect='sqlite', engine=Engine(sqlite:///:memory:), metadata=MetaData())

In [3]:
@newtable.table_schema
class MyContainer0:
    id: int
    name: str
    age: int

with core.tablemaker() as tmaker:
    tab0 = tmaker.new_table(container_type=MyContainer0)
pprint.pprint(core.inspect_columns_all())

{'MyContainer0': [{'default': None,
                   'name': 'age',
                   'nullable': True,
                   'primary_key': 0,
                   'type': INTEGER()},
                  {'default': None,
                   'name': 'id',
                   'nullable': True,
                   'primary_key': 0,
                   'type': INTEGER()},
                  {'default': None,
                   'name': 'name',
                   'nullable': True,
                   'primary_key': 0,
                   'type': VARCHAR()}]}


In [4]:
with core.query() as q:
    q.insert_multi(tab0, [
        {'name': 'Devin J. Cornell', 'age': 50},
        {'name': 'Dorothy Andrews', 'age': 49},
    ])
    print(q.select(tab0).all())

[(50, None, 'Devin J. Cornell'), (49, None, 'Dorothy Andrews')]


In [5]:
with tab0.query() as q:
    q.insert_single(MyContainer0(id=0, name='John Doe', age=30))
    print(q.select())

[MyContainer0(id=None, name='Devin J. Cornell', age=50), MyContainer0(id=None, name='Dorothy Andrews', age=49), MyContainer0(id=0, name='John Doe', age=30)]


In [6]:
# alternately you can use the core.connect() context manager 
# with the doctable factory method constructor
@newtable.table_schema
class MyContainer01:
    id: int
    name: str
    age: int

with core.connect() as conn:
    tab0 = newtable.DocTable.from_container(MyContainer01, core=core)
    conn.commit()
print(core.inspect_table_names())

['MyContainer0']


In [7]:
import datetime
@newtable.table_schema(table_name='mytable1')
class MyContainer1:
    name: str
    age: int
    id: int = newtable.Column(
        column_args=newtable.ColumnArgs(order=0, primary_key=True, autoincrement=True),
    )
    updated: datetime.datetime = newtable.Column(
        column_args=newtable.ColumnArgs(default=datetime.datetime.utcnow),
    )
    added: datetime.datetime = newtable.Column(
        field_args=newtable.FieldArgs(),
        column_args=newtable.ColumnArgs(
            default=datetime.datetime.utcnow, 
            onupdate=datetime.datetime.utcnow
        )
    )

TypeError: __init__() got an unexpected keyword argument 'order'

In [ ]:
# create a new table in metadata (does not create table in database)
tab0 = newtable.DocTable.from_container(MyContainer0, core=core)
print(tab0)
print(core.inspect_table_names())

# creates a new doctable with the same table metadata
tab_tmp = newtable.DocTable.from_container(MyContainer0, core=core, extend_existing=True)
print(tab_tmp.table is tab0.table)
print(core.inspect_table_names())
core.connect().commit()
print(core.inspect_table_names())


InvalidRequestError: Table 'MyContainer0' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [ ]:
tab1 = core.tablemaker().new_table(container_type=MyContainer0, extend_existing=True)
pprint.pprint(tab1)

DocTable(schema=TableSchema(table_name='MyContainer0', container_type=<class '__main__.MyContainer0'>, columns=[ColumnInfo(attr_name='id', type_hint=<class 'int'>, params=ColumnParams(column_name=None, sqlalchemy_type=<class 'sqlalchemy.sql.sqltypes.Integer'>, foreign_key=None, type_kwargs=None, column_kwargs={'primary_key': True, 'autoincrement': True}, field_kwargs={})), ColumnInfo(attr_name='name', type_hint=<class 'str'>, params=ColumnParams(column_name=None, sqlalchemy_type=None, foreign_key=None, type_kwargs={}, column_kwargs={}, field_kwargs=None)), ColumnInfo(attr_name='age', type_hint=<class 'int'>, params=ColumnParams(column_name=None, sqlalchemy_type=None, foreign_key=None, type_kwargs={}, column_kwargs={}, field_kwargs=None))], indices=[], constraints=[], table_kwargs={}), table=Table('MyContainer0', MetaData(), Column('id', Integer(), table=<MyContainer0>, primary_key=True, nullable=False), Column('name', String(), table=<MyContainer0>), Column('age', Integer(), table=<MyC